In [22]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# Choix des $q_k$ aléatoire

In [23]:
def next_matrix(Q, w0, x0, w, x, max_iter):
    N = len(x)
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[N, 0] = w0
    
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    for i in range(N):
        x_tilde[i+1] = x[i]

    it = 0
    while(it < max_iter):
        is_compatible = True
        q = np.random.randn(N)

        Q_tilde[0, 0] = - np.sum(q)
        for i in range(N):
            Q_tilde[0, i+1] = q[i]

        # vérification de la compatibilité
        Q_tilde_inv = np.linalg.inv(Q_tilde)
        
        # [Q_tilde Diag(x0,x1,...,xN) Q_tilde-1]ij <= 0 pour i != j
        D_tilde = np.diag(x_tilde)
        A = Q_tilde @ D_tilde @ Q_tilde_inv
        for i in range(N+1):
            for j in range(N+1):
                if(i != j and A[i, j] > 0):
                    is_compatible = False
        it += 1
        
        if(is_compatible):
            return Q_tilde
    return None

In [24]:
Q = np.array([[-1, 1],
              [1, 2]])

x = np.array([1, 10])
w = np.array([1, 2])
x0 = 0.5
w0 = 0.3

In [25]:
print(next_matrix(Q, w0, x0, w, x, 10000))

None


# Optimisation pour trouver des $q_k$ convenables

In [26]:
def compute_A(Q, w0, x0, x, q):
    N = len(x)
    
    # Construction de Q_tilde (matrice (N+1)x(N+1))
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[0, 0] = -np.sum(q)
    Q_tilde[0, 1:] = q
    Q_tilde[N, 0] = w0  # note : w n'est pas utilisé ici

    # Construction de x_tilde (vecteur N+1)
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    x_tilde[1:] = x

    try:
        Q_tilde_inv = np.linalg.inv(Q_tilde)
    except np.linalg.LinAlgError:
        raise ValueError("Q_tilde is not invertible.")

    # Calcul de A = Q_tilde @ diag(x_tilde) @ Q_tilde_inv
    D_tilde = np.diag(x_tilde)
    A = Q_tilde @ D_tilde @ Q_tilde_inv

    return A

def phi(q, Q, w0, x0, w, x, penalty=1e6):
    A = compute_A(Q, w0, x0, x, q)
    
    N = A.shape[0]
    total = 0.0
    for i in range(N):
        for j in range(N):
            if i != j:
                if A[i, j] > 0:
                    total += 50 * A[i, j]
                else:
                    total += A[i, j]
    return total

def optimize_q_sum1(Q, w0, x0, w, x, penalty=1e6):
    N = len(x)
    
    def objective(q):
        return phi(q, Q, w0, x0, w, x, penalty)
    
    cons = [
    {'type': 'ineq', 'fun': lambda q: -q[0]},  # q[0] <= 0 ⇒ -q[0] ≥ 0
    {'type': 'ineq', 'fun': lambda q: q[1]}    # q[1] ≥ 0
    ]
    
    q0 = np.random.randn(N)
    q0 = q0 / np.sum(q0)  # normaliser q0 pour respecter la contrainte au départ
    
    res = minimize(
        objective,
        x0=q0,
        method='SLSQP',  # méthode supportant contraintes
        constraints=cons,
        options={'disp': True}
    )
    
    if res.success:
        print("Optimisation réussie")
    else:
        print("Échec de l'optimisation :", res.message)
    
    return res.x, res.fun


q_opt, phi_val = optimize_q_sum1(Q, w0, x0, w, x)
print("q optimal :", q_opt)
print("phi(q_opt) :", phi_val)


Inequality constraints incompatible    (Exit mode 4)
            Current function value: -59177719737434.336
            Iterations: 5
            Function evaluations: 21
            Gradient evaluations: 5
Échec de l'optimisation : Inequality constraints incompatible
q optimal : [-9.98573635e-13  9.54791801e-13]
phi(q_opt) : -59177719737434.336


# À la main

In [98]:
def check_neg(Q_tilde, w0, x0, w, x):
    N = len(Q_tilde) - 1
    x_tilde = np.zeros(N + 1)
    x_tilde[0] = x0
    x_tilde[1:] = x
    D = np.diag(x_tilde)
    Q_tilde_inv = np.linalg.inv(Q_tilde)
    M = Q_tilde @ D @ Q_tilde_inv
    print(M)
    print('\n')
    for i in range(N+1):
        for j in range(N+1):
            if(i!=j and M[i,j] > 0):
                return False
    return True


In [99]:
N = 2
x = np.array([1, 10])
w = np.array([1, 2])
x0 = 8
w0 = -3

Q = np.array([[-2, 3],
              [w[0], w[1]]])

q1 = 1
q2 = -4

Q_tilde = np.zeros((N+1, N+1))
Q_tilde[1:, 1:] = Q
Q_tilde[N, 0] = w0
Q_tilde[0,1] = q1
Q_tilde[0,2] = q2
Q_tilde[0,0] = -q1-q2

print(Q_tilde)

print(check_neg(Q_tilde, w0, x0, w, x))

[[ 3.  1. -4.]
 [ 0. -2.  3.]
 [-3.  1.  2.]]
[[-10.5 -15.  -18.5]
 [ 27.   28.   27. ]
 [ -6.5  -3.    1.5]]


False


In [ ]:
def test_random_q(Q, w0, x0, x, condition_fn, trials=100):
    """
    Teste 100 valeurs aléatoires de (q1, q2) telles que :
    q1 >= 0, q2 <= 0

    - Q : matrice de base (taille NxN)
    - w0, x0, x : paramètres pour construire Q_tilde
    - condition_fn : fonction prenant Q_tilde et retournant True/False
    - trials : nombre d'essais

    Affiche les cas où la condition est satisfaite.
    """
    N = len(x)
    results = []

    for _ in range(trials):
        q1 = np.random.rand()  # q1 ∈ [0, 1]
        q2 = -np.random.rand()  # q2 ∈ [-1, 0]
        q = np.array([q1, q2])

        # Construction de Q_tilde
        Q_tilde = np.zeros((N + 1, N + 1))
        Q_tilde[1:, 1:] = Q
        Q_tilde[0, 0] = -np.sum(q)
        Q_tilde[0, 1:] = q
        Q_tilde[N, 0] = w0

        try:
            if condition_fn(Q_tilde,  w0, x0, w, x):
                results.append((q1, q2))
        except np.linalg.LinAlgError:
            continue  # Matrice non inversible, on passe

    print(f"Nombre de matrices satisfaisant la condition : {len(results)}")

    return results

Q = np.array([[-1, 1], [0.5, 1.0]])
w0 = 1.0
x0 = 1.0
x = np.array([1.0, 10])

test_random_q(Q, w0, x0, x, check_neg, trials=100)


[[ -3.37250729   6.16781471  -3.75030416]
 [ 17.62563028 -23.86253644  15.11752186]
 [ 35.25126056 -49.72507288  31.23504373]]


[[ 1.68839252 -0.10812337  0.01594528]
 [-2.81385732  1.44196258 -0.06517755]
 [-5.62771464  0.88392516  0.8696449 ]]


[[ 1.60479012 -0.00849441 -0.06583607]
 [-1.70525012  1.02395062  0.18562963]
 [-3.41050024  0.04790124  1.37125926]]


[[ 1.60572325 -0.04975088 -0.31786038]
 [-0.34840624  1.02861623  0.18283026]
 [-0.69681248  0.05723246  1.36566052]]


[[ 1.72737659 -0.69552066  0.19889843]
 [-0.66605317  1.63688294 -0.18212977]
 [-1.33210634  1.27376589  0.63574047]]


[[ 1.5773558   0.19249141 -0.45552256]
 [-0.33959337  0.88677899  0.2679326 ]
 [-0.67918674 -0.22644202  1.53586521]]


[[ 2.37240658 -1.72512252  0.94573597]
 [-3.0724075   4.8620329  -2.11721974]
 [-6.144815    7.7240658  -3.23443948]]


[[  2.5746587   -1.40650578   0.78618046]
 [ -5.45591361   5.8732935   -2.7239761 ]
 [-10.91182723   9.74658701  -4.4479522 ]]


[[ 1.60469461 -0.04493

[]